In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY="5g"
spark = SparkSession.builder.appName("taxi-fare-prediciton")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 13:23:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 전처리 된 데이터 불러오기

In [2]:
# 분산 저장 되어서 파티션으로 분리된 파일 불러오기
save_dir="/home/ubuntu/working/spark/data/ml-data"

train_sdf = spark.read.parquet(f"{save_dir}/train/")
test_sdf  = spark.read.parquet(f"{save_dir}/test/")

In [3]:
train_sdf.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [4]:
test_sdf.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



# 파이프라인 구성

In [5]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler, StandardScaler

stages = []

cat_features = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

for c in cat_features:
    cat_indexer = StringIndexer(inputCol=c, outputCol=c+"_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()],
                                   outputCols=[c+"_onehot"])
    
    stages += [cat_indexer, onehot_encoder]

num_features = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_features:
    num_assembler = VectorAssembler(inputCols=[n], outputCol=n+"_vector")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=n+"_scaled")
    
    stages += [num_assembler, num_scaler]

assembler_inputs = [c + "_onehot" for c in cat_features] + [n + "_scaled" for n in num_features]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="feature_vector")
stages += [assembler]

stages

[StringIndexer_2ab34b162d65,
 OneHotEncoder_3144fe1e91bd,
 StringIndexer_fbcecd7a32b4,
 OneHotEncoder_383b6ad54d86,
 StringIndexer_1d6110ec2a3f,
 OneHotEncoder_60a758c02c6e,
 VectorAssembler_90975d3b1aa5,
 StandardScaler_5d046923be1d,
 VectorAssembler_5049ce9da408,
 StandardScaler_60b6b8239aa1,
 VectorAssembler_17f62a15ef41,
 StandardScaler_a044bf167ae1,
 VectorAssembler_3b1e4b6931a3]

# 하이퍼 파라미터 튜닝
`GridSearch` + `Cross Validation` 을 수행하여 최적의 하이퍼 파라미터 얻어내기

In [6]:
# 모델 까지를 파이프라인에 넣고, 모델이 추가된 파이프라인을 그리드 서치에 사용
from pyspark.ml.regression import LinearRegression

# 모델 생성
lr = LinearRegression(
    maxIter=30,
    solver='normal',
    labelCol = 'total_amount',
    featuresCol = 'feature_vector'
)

# LinearRegression 모델 까지가 하나의 파이프라인이 된다.
cv_stages = stages + [lr]
cv_stages

[StringIndexer_2ab34b162d65,
 OneHotEncoder_3144fe1e91bd,
 StringIndexer_fbcecd7a32b4,
 OneHotEncoder_383b6ad54d86,
 StringIndexer_1d6110ec2a3f,
 OneHotEncoder_60a758c02c6e,
 VectorAssembler_90975d3b1aa5,
 StandardScaler_5d046923be1d,
 VectorAssembler_5049ce9da408,
 StandardScaler_60b6b8239aa1,
 VectorAssembler_17f62a15ef41,
 StandardScaler_a044bf167ae1,
 VectorAssembler_3b1e4b6931a3,
 LinearRegression_d2897a1d14e8]

##  파이프라인 생성

In [7]:
from pyspark.ml import Pipeline

cv_pipeline = Pipeline(stages=cv_stages)
cv_pipeline

Pipeline_f15d383d2f17

# GridSearch 및 Cross Validation 설정

In [8]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

# GridSearch를 위한 하이퍼 파라미터 정의
param_grid = (
    ParamGridBuilder()
        .addGrid(lr.elasticNetParam, [0.1, 0.2, 0.3, 0.4, 0.5])
        .addGrid(lr.regParam, [0.01, 0.02, 0.03, 0.04, 0.05])
        .build()
)

In [13]:
# CrossValidator 정의
cross_val = CrossValidator(
    estimator=cv_pipeline, # estimator : 추정기. 여기에서는 파이프라인 자체가 마지막에 추정(예측) ,# GridSearch를 수행할 파라미터가 들어있는 ParamGridBuilder을 한다.
    estimatorParamMaps=param_grid, ,# GridSearch를 수행할 파라미터가 들어있는 ParamGridBuilder
    evaluator=RegressionEvaluator(labelCol="total_amount"),
    numFolds=5 # 폴드의 개수
)

In [10]:
# 임의의 데이터 세트를 생성. 전체로 다 하면 시간이 너무 많이 걸릴거 같아요...ㅠㅠ

# 전체 훈련 데이터에서 0.001%만 사용
toy_df = train_sdf.sample(False, 0.001, seed=1)
toy_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [14]:
cv_model = cross_val.fit(toy_df)

24/04/01 13:34:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/01 13:34:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [16]:
# 성능이 가장 좋았던 모델 가져오기, 파이프라인의 제일 마지막이 모델!
best_model = cv_model.bestModel.stages[-1]
best_model

LinearRegressionModel: uid=LinearRegression_d2897a1d14e8, numFeatures=288

In [17]:
# 모델로부터 최고의 성능을 냈던 하이퍼 파라미터 추출
alpha = best_model._java_obj.getElasticNetParam()
reg_param = best_model._java_obj.getRegParam()

In [19]:
alpha, reg_param

(0.5, 0.05)

# 최적의 성능을 냈던 파라미터를 이용해 모델 재훈련
- 이미 최적의 하이퍼 파라미터를 구해 냈으니 전처리 파이프라인만을 이용해 모델을 하나만 얻어낼 생각

In [20]:
# 데이터 변환 파이프라인을 따로 정의
pipeline = Pipeline(stages=stages)

fitted_transformer = pipeline.fit(train_sdf)

vec_train_sdf = fitted_transformer.transform(train_sdf)
vec_train_sdf.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nul

In [21]:
lr = LinearRegression(
    maxIter=50,
    solver='normal',
    labelCol='total_amount',
    featuresCol = "feature_vector",
    
    # GridSearch를 통해 얻어낸 하이퍼 파라미터를 직접 설정
    elasticNetParam=alpha,
    regParam=reg_param
)

lr_model = lr.fit(vec_train_sdf)

24/04/01 13:51:55 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/04/01 13:51:55 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [23]:
# 테스트 데이터 예측
vec_test_sdf = fitted_transformer.transform(test_sdf)

predictions = lr_model.transform(vec_test_sdf)
predictions.cache()

DataFrame[passenger_count: int, pickup_location_id: int, dropoff_location_id: int, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, passenger_count_vector: vector, passenger_count_scaled: vector, trip_distance_vector: vector, trip_distance_scaled: vector, pickup_time_vector: vector, pickup_time_scaled: vector, feature_vector: vector, prediction: double]

In [24]:
lr_model.summary.rootMeanSquaredError

3.2936999070427677

In [25]:
lr_model.summary.r2

0.7912653136612472

# 튜닝 모델 저장

In [26]:
model_dir = "/home/ubuntu/working/spark/data/ml-data/model/"

In [27]:
lr_model.save(model_dir)

# 저장된 모델 불러오기

In [28]:
# 저장된 모델을 불러올 때 저장한 모델의 클래스를 반드시 불러와야 함!!!
from pyspark.ml.regression import LinearRegressionModel

lr_model_load = LinearRegressionModel().load(model_dir)

In [29]:
# 불러온 모델로 예측
lr_model_load.transform(vec_test_sdf).show(5)

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|pickup_location_id_idx|pickup_location_id_onehot|dropoff_location_id_idx|dropoff_location_id_onehot|day_of_week_idx|day_of_week_onehot|passenger_count_vector|passenger_count_scaled|trip_distance_vector|trip_distance_scaled|pickup_time_vector|  pickup_time_scaled|      feature_vector|        prediction|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+----------------------

In [30]:
spark.stop()